# Max Flow Applications

The purpose of this assignment is to investigate applications of finding a Max Flow. The problem asks you to design and implement an algorithm for shipping a material between nodes with different supply and demand requirements.

* Please write code *only* in the bodies of the two functions, that is, following the TODO comments.
* Be careful not to use varibles defined outside of the functions.
* Breaking the two above rules may lead to 0 grades.

## Movie distribution

First solve Problem 3 from HW3-theoretical. 

Now suppose a movie distributor would like to ship a copy of a film from CA to every other state. There are therefore 48 units to ship out of CA, and each other state receives 1 unit. 

The dataset contiguous-usa.dat lists the adjacent states in the US. Each line lists two adjacent states; thus AK and HI are omitted, but DC is included in the data. The following code reads in the graph of US states.

In [1]:
import networkx as nx
G = nx.Graph()

usa = open('contiguous-usa.dat')
for line in usa:
    s1, s2 = line.strip().split()
    G.add_edge(s1, s2)    

We now encode the demands into the graph.

In [2]:
for state in G.nodes():
    if state != 'CA':
        G.node[state]['demand'] = 1
G.node['CA']['demand'] = -48

We will assign a uniform capacity of 16 to each edge. Since CA has only three adjacent states, this is the smallest possible uniform capacity that allows one to ship all 48 units out of CA. As we have created an undirected graph, and flows have directions, we first convert the graph to a directed graph.

In [3]:
G = nx.DiGraph(G)
uniform_capacity = 16
for (s1, s2) in G.edges():
    G.edge[s1][s2]['capacity'] = uniform_capacity

Complete the following function to implement your algorithm to find a flow with demands. Your function should work correctly for any input, not just the movie instance considered here. As always, you are encouraged to define auxiliary functions as needed for clarity.

In [4]:
def flow_with_demands(graph):
    """Computes a flow with demands over the given graph.
    
    Args:
        graph: A directed graph with nodes annotated with 'demand' properties and edges annotated with 'capacity' 
            properties.
        
    Returns:
        A dict of dicts containing the flow on each edge. For instance, flow[s1][s2] should provide the flow along
        edge (s1, s2).
        
    Raises:
        NetworkXUnfeasible: An error is thrown if there is no flow satisfying the demands.
    """
    # TODO: Implement the function.
    
    # Reduction
    reduction = graph.copy()
    reduction.add_node('Supersource') # supersource
    reduction.add_node('Supersink') # supersink        

    # Add edges to supersink with proper capacities
    for n in graph.nodes():
        if graph.node[n]['demand'] > 0:
            reduction.add_edge(n, 'Supersink')
            reduction.edge[n]['Supersink']['capacity'] = graph.node[n]['demand']
                
    # Add edges from supersource with proper capacities
    for n in graph.nodes():
        if graph.node[n]['demand'] < 0:
            reduction.add_edge('Supersource', n)
            reduction.edge['Supersource'][n]['capacity'] = -graph.node[n]['demand']
        
    # Sum demands for all sinks for checking flow feasibility
    sink_demand = sum(graph.node[n]['demand'] for n in graph.nodes() if graph.node[n]['demand'] > 0)
    
    if nx.maximum_flow_value(reduction, 'Supersource', 'Supersink') != sink_demand:
        raise nx.NetworkXUnfeasible("There is no flow satisfying the demands.")
            
    # Run max flow algorithm on reduction and return flow
    circulation = nx.maximum_flow(reduction, 'Supersource', 'Supersink')

    flows = {}
    
    # Remove entries for Supersource and Supersink     
    for u in circulation[1]:
        if u != 'Supersource' and u != 'Supersink':
            flows[u] = {}        
            for v in circulation[1][u]:
                if v != 'Supersink':
                    flows[u][v] = circulation[1][u][v]
                    
    return flows


To verify that your solution is correct, implement a function that computes the total flow into each node (which will be negative for supply nodes).

In [5]:
def divergence(flow):
    """Computes the total flow into each node according to the given flow dict.
    
    Args:
        flow: the flow dict recording flow between nodes.
        
    Returns:
        A dict of the net flow into each node.
    """
    # TODO: Implement the function.

    net_flow = {}
    
    # Default to 0     
    for u in flow:
        for v in flow[u]:
            net_flow[v] = 0
    
    # Add inflows
    for u in flow:
        for v in flow[u]:
            net_flow[v] = net_flow[v] + flow[u][v]
    
    # Subtract outflows
    for u in flow:
        for v in flow[u]:
            net_flow[u] = net_flow[u] - flow[u][v]

    return net_flow

The following code performs a sanity check on your function (but does not completely confirm correctness).

In [6]:
flow = flow_with_demands(G)
div = divergence(flow)
print "Flow satisfies all demands:", all(div[n] == G.node[n]['demand'] for n in G.nodes())

Flow satisfies all demands: True
